<a href="https://colab.research.google.com/github/askolesov0608/SJK006-Project/blob/main/Copy_of_SJK006_RF_GB_LR_SVM_CroosValid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

# Загрузка данных
file_path = 'TestDataCV_DigitalV2.xlsx'
data = pd.read_excel(file_path)

# Создание новой целевой переменной для должности
positions = ['Human Resources', 'Research & Development', 'Sales',
             'Life Sciences', 'Marketing', 'Medical', 'Other']
data['Position'] = (data[positions] == 1).dot(range(1, len(positions) + 1))

# Выбор признаков для модели
features_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
                    'HourlyRate', 'JobInvolvement', 'JobLevel', 'GenderM', 'GenderF',
                    'Single', 'Married', 'Divorced']
features = data[features_columns]
target = data['Position']

# Бутстрэппинг каждого класса до 1000 записей
bootstrapped_data = pd.DataFrame(columns=features_columns + ['Position'])
for position in target.unique():
    position_data = data[data['Position'] == position]
    bootstrapped_sample = resample(position_data, replace=True, n_samples=1000, random_state=42)
    bootstrapped_data = pd.concat([bootstrapped_data, bootstrapped_sample])

# Подготовка данных для обучения
X = bootstrapped_data[features_columns]
y = bootstrapped_data['Position'].astype(int)  # Убедитесь, что y - это массив целых чисел

# Масштабирование признаков
X_scaled = StandardScaler().fit_transform(X)

# Определение моделей для сравнения
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "SVM": SVC(random_state=42)
}

# Кросс-валидация для каждой модели
cv_scores = {}
for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=10)
    cv_scores[name] = scores
    print(f"Модель: {name}")
    print(f"Средняя точность: {scores.mean()}, Стандартное отклонение: {scores.std()}")

Модель: Random Forest
Средняя точность: 0.9779, Стандартное отклонение: 0.00480520551069359
Модель: Gradient Boosting
Средняя точность: 0.8181, Стандартное отклонение: 0.010348429832588097
Модель: Logistic Regression
Средняя точность: 0.29009999999999997, Стандартное отклонение: 0.01325481044753186
Модель: SVM
Средняя точность: 0.7748, Стандартное отклонение: 0.012718490476467729
